In [1]:
import open3d as o3d
import numpy as np
from scipy import interpolate

# PLY 파일 경로 설정
ply_file_path = "C:/Users/SKH/Github_local/3D_Modeling_with_Three.js/pointcloudprocessing/data/point_cloud (4).ply"

# 포인트 클라우드 데이터를 읽어옵니다.
pcd = o3d.io.read_point_cloud("C:/Users/SKH/Github_local/3D_Modeling_with_Three.js/pointcloudprocessing/data/point_cloud (4).ply")

# 포인트 클라우드에서 포인트들의 좌표를 추출합니다.
points = np.asarray(pcd.points)

# 좌표를 출력합니다.
for point in points:
    print(f"Point: {point}")

# 포인트 클라우드 데이터 읽기
pcd = o3d.io.read_point_cloud(ply_file_path)
print(pcd)  # 입력 포인트 클라우드 확인

# 처음 입력 포인트 클라우드 시각화
o3d.visualization.draw_geometries([pcd])

ModuleNotFoundError: No module named 'open3d'

In [ ]:
# Statistical Outlier Removal 이용한 이상치 제거
cl, ind = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
inlier_pcd = pcd.select_by_index(ind)
print("Statistical Outlier Removal")
o3d.visualization.draw_geometries([inlier_pcd])

In [ ]:
# 데이터 다운샘플링
downsampled_pcd = inlier_pcd.voxel_down_sample(voxel_size=0.05)
print("Downsampled")
o3d.visualization.draw_geometries([downsampled_pcd])

In [ ]:
# 스플라인 보간
downsampled_points = np.asarray(downsampled_pcd.points)
x = downsampled_points[:, 0]
y = downsampled_points[:, 1]
z = downsampled_points[:, 2]

# 스플라인 함수 설정
tck, _ = interpolate.splprep([x, y, z], s=0)
u = np.linspace(0, 1, num=1000)
interpolated_points = interpolate.splev(u, tck)

interpolated_pcd = o3d.geometry.PointCloud()
interpolated_pcd.points = o3d.utility.Vector3dVector(np.column_stack(interpolated_points))
print("Interpolated")
o3d.visualization.draw_geometries([interpolated_pcd])

In [ ]:
# 압축 작업 (예: 랜덤 샘플링)
sample_size = min(1000, len(interpolated_points[0]))  # 원본 배열 크기보다 큰 샘플 크기를 방지
sampled_indices = np.random.choice(len(interpolated_points[0]), size=sample_size, replace=False)
compressed_points = np.array([interpolated_points[0][sampled_indices], interpolated_points[1][sampled_indices], interpolated_points[2][sampled_indices]]).T
compressed_pcd = o3d.geometry.PointCloud()
compressed_pcd.points = o3d.utility.Vector3dVector(compressed_points)
print("Compressed")
o3d.visualization.draw_geometries([compressed_pcd])

In [ ]:
# 압축된 데이터를 PLY 파일로 저장
output_ply_file = "C:/Users/SKH/Github_local/3D_Modeling_with_Three.js/pointcloudprocessing/data/compressed_point_cloud.ply"
o3d.io.write_point_cloud(output_ply_file, compressed_pcd)

print(f"Compressed point cloud saved to {output_ply_file}")
